In [1]:
import os
import json
import jsonlines
import pandas as pd

In [2]:
os.chdir(os.path.join(os.getcwd(), 'data'))

df = pd.read_excel('markers.xlsx')
df_board = pd.read_excel('board.xlsx')
df.shape

(109, 14)

## 原始数据

In [3]:
df.head()

,type,name,short_name,desc,logo,images,phone,address,panorama,location,icon,position,scale,_id
0,景点,孙中山铜像,铜像,地点描述,NaN,2.0,NaN,地址,2291779.0,"23.634548,113.676900",tx@2,0.0,16.0,5ce8fe1c29c7a8581bc1e98b
1,景点,校训石,NaN,地点描述,NaN,1.0,NaN,NaN,NaN,"23.634037,113.677393",NaN,NaN,NaN,NaN
2,景点,南芳湖,NaN,地点描述,NaN,2.0,NaN,NaN,NaN,"23.629113,113.679507",NaN,NaN,NaN,NaN
3,景点,西区公园,NaN,地点描述,NaN,2.0,NaN,NaN,NaN,"23.629958,113.677726",NaN,NaN,NaN,NaN
4,景点,大礼堂,NaN,地点描述,NaN,3.0,NaN,NaN,NaN,"23.631772,113.679662",NaN,NaN,NaN,NaN


In [4]:
df[['images', 'panorama', 'position']] = df[['images', 'panorama', 'position']].fillna(0).astype('int')
df.head()

,type,name,short_name,desc,logo,images,phone,address,panorama,location,icon,position,scale,_id
0,景点,孙中山铜像,铜像,地点描述,NaN,2,NaN,地址,2291779,"23.634548,113.676900",tx@2,0,16.0,5ce8fe1c29c7a8581bc1e98b
1,景点,校训石,NaN,地点描述,NaN,1,NaN,NaN,0,"23.634037,113.677393",NaN,0,NaN,NaN
2,景点,南芳湖,NaN,地点描述,NaN,2,NaN,NaN,0,"23.629113,113.679507",NaN,0,NaN,NaN
3,景点,西区公园,NaN,地点描述,NaN,2,NaN,NaN,0,"23.629958,113.677726",NaN,0,NaN,NaN
4,景点,大礼堂,NaN,地点描述,NaN,3,NaN,NaN,0,"23.631772,113.679662",NaN,0,NaN,NaN


## 目标数据格式
```
{
    type: '景点',
    scale: 15.0,
    icon: 'jt',
    postion: 0,
    data:   [
        {
            name: "孙中山铜像",
            short_name: "铜像", 
            desc: "中山铜像...", 
            logo: "tx", 
            icon: "tx@2",
            images: 3,
            panorama: 0, 
            latitude: "23.635875",
            longitude: "113.678965",
            contact: { phone: "020-123456", address: "出门左转" }
         },
             ...
         {...}
      ]
}
```

In [5]:
# 填充NaN值
df.fillna('', inplace=True)

In [6]:
# location
df['latitude'] = df.apply(lambda x:x.location.split(',')[0], axis=1)
df['longitude'] =df.apply(lambda x:x.location.split(',')[1], axis=1)
df.drop('location', axis=1, inplace=True)
df.tail()

,type,name,short_name,desc,logo,images,phone,address,panorama,icon,position,scale,_id,latitude,longitude
104,校区,中山大学南方学院,南苑,中山大学南方学院是2006年经教育部批准，由中山大学与广东珠江投资集团合作创办的独立学院，是...,nfsysu,2,020-61787201,广州市从化区温泉大道882号中山大学南方学院,2291778,,11,,Rok4jKCtqaeJQmkYMlvk1G30I46gM0TEpKX9c8DziJZzWewg,23.632674,113.679404
105,探险,校长别墅,,地点描述,,1,,,0,,10,,wkNEmpv7L4Th8yfd1Dl34gDzSLlnmLFGF16ACmQ7fIZAMzzV,23.635659,113.678386
106,探险,后山,,地点描述,,1,,,0,climb,0,,,23.632258,113.673241
107,探险,大礼堂之巅,天台风凉,地点描述,,1,,,0,,0,,,23.631629,113.679914
108,探险,枫林晚,,锦衣夜游好去处,,1,,,0,,0,,,23.636455,113.680172


In [7]:
# contact
df['contact'] = df.apply(lambda x:{
    'phone': x.phone,
    'address': x.address
}, axis=1)
df.drop(['phone', 'address'], axis=1, inplace=True)
df.tail()

,type,name,short_name,desc,logo,images,panorama,icon,position,scale,_id,latitude,longitude,contact
104,校区,中山大学南方学院,南苑,中山大学南方学院是2006年经教育部批准，由中山大学与广东珠江投资集团合作创办的独立学院，是...,nfsysu,2,2291778,,11,,Rok4jKCtqaeJQmkYMlvk1G30I46gM0TEpKX9c8DziJZzWewg,23.632674,113.679404,"{'phone': '020-61787201', 'address': '广州市从化区温泉..."
105,探险,校长别墅,,地点描述,,1,0,,10,,wkNEmpv7L4Th8yfd1Dl34gDzSLlnmLFGF16ACmQ7fIZAMzzV,23.635659,113.678386,"{'phone': '', 'address': ''}"
106,探险,后山,,地点描述,,1,0,climb,0,,,23.632258,113.673241,"{'phone': '', 'address': ''}"
107,探险,大礼堂之巅,天台风凉,地点描述,,1,0,,0,,,23.631629,113.679914,"{'phone': '', 'address': ''}"
108,探险,枫林晚,,锦衣夜游好去处,,1,0,,0,,,23.636455,113.680172,"{'phone': '', 'address': ''}"


In [8]:
import pinyin

def get_pinyin_first_alpha(name):
    '''
        根据type中文名转为拼音缩写，用以匹配icon
    '''
    return "".join([i[0] for i in pinyin.get(name, " ").split(" ")])

In [9]:
data = [{'type': t, 'icon': get_pinyin_first_alpha(t), 'data': []} for t in set(df.type)] 
data

[{'type': '交通', 'icon': 'jt', 'data': []},
 {'type': '食堂', 'icon': 'st', 'data': []},
 {'type': '体育场所', 'icon': 'tycs', 'data': []},
 {'type': '校区', 'icon': 'xq', 'data': []},
 {'type': '学生宿舍', 'icon': 'xsss', 'data': []},
 {'type': '生活服务', 'icon': 'shfw', 'data': []},
 {'type': '行政楼', 'icon': 'xzl', 'data': []},
 {'type': '校门', 'icon': 'xm', 'data': []},
 {'type': '教学楼', 'icon': 'jxl', 'data': []},
 {'type': '教工宿舍', 'icon': 'jgss', 'data': []},
 {'type': '景点', 'icon': 'jd', 'data': []},
 {'type': '探险', 'icon': 'tx', 'data': []}]

In [10]:
for index, row in df.iterrows():
    for i in data:
        if i['type'] == row.type:
            i['data'].append(dict(row))
            if row.scale != '':
                i['scale'] = i['data'][0]['scale']
            if row.position != '':
                i['position'] = i['data'][0]['position']
            if row._id != '':
                i['_id'] = i['data'][0]['_id']
data

[{'type': '交通',
  'icon': 'jt',
  'data': [{'type': '交通',
    'name': '卫东(公交站)',
    'short_name': '卫东站',
    'desc': '地点描述',
    'logo': '',
    'images': 0,
    'panorama': 0,
    'icon': 'station',
    'position': 3,
    'scale': 15.0,
    '_id': '5ce8fe1c29c7a8581bc1e991',
    'latitude': '23.644299',
    'longitude': '113.671715',
    'contact': {'phone': '', 'address': ''}},
   {'type': '交通',
    'name': '校车站',
    'short_name': '',
    'desc': '地点描述',
    'logo': '',
    'images': 2,
    'panorama': 0,
    'icon': 'station',
    'position': 0,
    'scale': '',
    '_id': '',
    'latitude': '23.6352853977',
    'longitude': '113.6773502827',
    'contact': {'phone': '', 'address': ''}},
   {'type': '交通',
    'name': '停车场（铜像）',
    'short_name': '',
    'desc': '地点描述',
    'logo': '',
    'images': 1,
    'panorama': 0,
    'icon': '',
    'position': 0,
    'scale': '',
    '_id': '',
    'latitude': '23.634622',
    'longitude': '113.676449',
    'contact': {'phone': '', 'addre

# Board 处理

## 目标数据格式
```
{
    'type': '信息类型',
    'data': [
        {
            'time': 123456,   # timestamp
            'content': '信息内容'
        },
        ...
        {...}
    ]
}
```

In [11]:
df_board = df_board[df_board.status != 1]
df_board.drop(columns='status', inplace=True)
df_board.head()

,type,time,content
0,校园动态,123456,测试信息
1,校园动态,123456,测试信息2
2,讲座论坛,123456,富文本信息


In [12]:
data_board = [{'type': t, 'data': []} for t in set(df_board.type)] 
for itme in data_board:
    for index, row in df_board.iterrows():
        if row.type == itme['type']:
            itme['data'].append(dict(row))
data_board

[{'type': '校园动态',
  'data': [{'type': '校园动态', 'time': 123456, 'content': '测试信息'},
   {'type': '校园动态', 'time': 123456, 'content': '测试信息2'}]},
 {'type': '讲座论坛',
  'data': [{'type': '讲座论坛', 'time': 123456, 'content': '富文本信息'}]}]

## 输出JSON
需要注意以下几点：

1. JSON 数据不是数组，而是类似 **[JSON Lines](http://jsonlines.org/)**，即各个记录对象之间使用 \n 分隔，而非逗号；

2. JSON 数据每个键值对的键名首尾不能是 .，例如 ".a"、"abc."，且不能包含多个连续的 .，例如 "a..b"；

3. 键名不能重复，且不能有歧义，例如 {"a": 1, "a": 2} 或 {"a": {"b": 1}, "a.b": 2}；

4. 时间格式须为 ISODate 格式，例如 "date": { "$date" : "2018-08-31T17:30:00.882Z" }；

5. 当使用 Insert 冲突处理模式时，同一文件不能存在重复的 _id 字段，或与数据库已有记录相同的 _id 字段；

6. CSV 格式的数据默认以第一行作为导入后的所有键名，余下的每一行则是与首行键名一一对应的键值记录。

参考链接：
* https://developers.weixin.qq.com/miniprogram/dev/wxcloud/guide/database/import.html?search-key=JSON%20lines  

In [13]:
with jsonlines.open('data.json', mode='w') as writer:
    for i in data:
        writer.write(i)
        
with jsonlines.open('data_board.json', mode='w') as writer:
    for i in data_board:
        writer.write(i)

In [14]:
with open('data.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(data, separators='\n', ensure_ascii=False))

ValueError: not enough values to unpack (expected 2, got 1)